In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from encode_data import *
from midi_data import *

In [5]:
from tqdm import tqdm

## Encode music21 stream to text representation and back

### Load midi data

In [6]:
path = Path('data/midi')
csv_path = path/'metadata'

In [7]:
import pandas as pd

In [9]:
df = pd.read_csv(path/'metadata/combined.csv', index_col=0); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,13,14,15,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_time_signature,inferred_key,inferred_keyc,instruments,...,midi_keyc,midi_title,original_path,parts,seconds,section,song_url,source,time_signature,title
0,wayne-sharpe,128.0,NaN,128.0,C,1.0,4.0,C major,C major,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/wa...,yu-gi-oh3,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",15.000000,chorus,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,4/4,yu-gi-oh-theme-song
1,weezer,NaN,NaN,128.0,C,NaN,4.0,NaN,NaN,NaN,...,NaN,My New Song,data/midi/midi_sources/hooktheory/pianoroll/w/...,intro-and-verse,NaN,intro-and-verse,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,NaN,beverly-hills
2,wayne-sharpe,85.0,NaN,85.0,C,1.0,3.0,C major,C major,Piano,...,data/midi/transposed/hooktheory/pianoroll/w/wa...,yu-gi-oh,data/midi/midi_sources/hooktheory/pianoroll/w/...,"intro,chorus",25.411765,intro,https://www.hooktheory.com/theorytab/view/wayn...,hooktheory,3/4,yu-gi-oh-theme-song
3,woodentoaster,197.0,"Alt-Country,Hip-Hop/Rap",197.0,D,6.0,4.0,D minor,D minor,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/wo...,Nightmare Night,data/midi/midi_sources/hooktheory/pianoroll/w/...,"chorus,instrumental",9.746193,instrumental,https://www.hooktheory.com/theorytab/view/wood...,hooktheory,4/4,nightmare-night
4,weezer,93.0,Rock,93.0,A,1.0,4.0,F# minor,F# minor,"Piano,Piano",...,data/midi/transposed/hooktheory/pianoroll/w/we...,Weezer - Fall Together,data/midi/midi_sources/hooktheory/pianoroll/w/...,chorus,10.322581,chorus,https://www.hooktheory.com/theorytab/view/weez...,hooktheory,4/4,fall-together-


In [10]:
ygo = df.midi_keyc.iloc[0]

In [11]:
enr = 'data/midi/transposed/freemidi/genre-dance-eletric/Enrique Iglesias - Experiencia Religiosa.mid'

In [12]:
df.loc[df.midi_keyc == enr]

,artist,bpm,genres,ht_bpm,ht_key,ht_mode,ht_time_signature,inferred_key,inferred_keyc,instruments,...,midi_keyc,midi_title,original_path,parts,seconds,section,song_url,source,time_signature,title
20,Enrique Iglesias,114.0,dance,NaN,NaN,NaN,NaN,F major,F major,"Flute,Piano,Electric Bass,Electric Organ,Voice",...,data/midi/transposed/freemidi/genre-dance-elet...,NaN,NaN,NaN,244.210526,NaN,NaN,freemidi,4/4,Experiencia Religiosa


In [13]:
df_filtered = df.loc[df['time_signature'] == '4/4'][18000:19000]; df_filtered.shape

(1000, 21)

In [14]:
from data_sources import process_parallel

### Need better midi formatting

In [72]:
def is_channel(t, c_val):
    return any([c == c_val for c in t.getChannels()])

In [73]:
def has_event_type(t, etypes):
    if isinstance(etypes, list): etypes = set(etypes)
    elif isinstance(etypes, set): pass
    else: etypes = set([etypes])
    for e in t.events:
        if e.type in etypes:
            return True
    return False

In [74]:
from enum import Enum
class Track(Enum):
    INFO = 0
    MELODY = 1
    PIANO = 2 # discrete instruments - keyboard, woodwinds
    STRING = 3 # continuous instruments with pitch bend: violin, trombone, synths
    PERC = 4
    UNDEF = 5

In [75]:
INFO_TYPES = set(['TIME_SIGNATURE', 'KEY_SIGNATURE', 'SET_TEMPO'])
def is_info_track(t, idx):
    is_ch0 = idx == 0
    is_info = has_event_type(t, INFO_TYPES)
    if is_info and t.hasNotes(): raise Exception('Error: found track with notes and track info')
    if is_ch0 ^ is_info: raise Exception('Error: info channel is not channel 0')
    return is_info

In [87]:
def is_melody(t, fp=''):
    if not t.hasNotes(): return False
    # special case for hooktheory
    if 'hooktheory' in str(fp):
        if is_channel(t, 1): return True
        return False
    if has_event_type(t, ['LYRIC']): return True # lyrics associated with vocals

In [88]:
def is_percussion(t):
    return is_channel(t, 10)

In [89]:
def is_string(t):
    if not t.hasNotes(): return False
    if has_event_type(t, ['PITCH_BEND']): return True
    return False

In [90]:
def is_piano(t, fp):
    if not t.hasNotes(): return False
    if is_melody(t, fp): return False
    if is_string(t): return False
    if is_percussion(t): return False
    return True

In [91]:
def get_track_type(t, idx, fp=''):
    if is_info_track(t, idx): return Track.INFO
    if not t.hasNotes(): return Track.UNDEF
    if is_melody(t, fp): return Track.MELODY
    if is_percussion(t): return Track.PERC
    if is_string(t): return Track.STRING
    return Track.PIANO

In [92]:
def change_track_instrument(t, value):
    for idx,e in enumerate(t.events):
        if e.type == 'PROGRAM_CHANGE':
            e.data = value

In [170]:
def print_music21_instruments():
    for i in range(200):
        try: print(i, music21.instrument.instrumentFromMidiProgram(i))
        except: pass

type2inst = {
#     Track.MELODY: 52, # Voice
    Track.MELODY: 79, # Ocarina
    Track.STRING: 24, # Guitar
    Track.PIANO: 0 # Piano
}

In [ ]:
# from hooktheory offset estimation
#     mode = metadata['ht_mode']
#     if mode is None:
#         print('No mode found. Assuming cmajor', fp)
#         mode = 'major'
#     offset = keyc_offset(metadata['ht_key'], mode)
    
    
# def get_music21_attr(fp, transpose=True, offset=None):
#     try:
#         stream = music21.converter.parse(fp)
#     except Exception as e:
#         print('Could not parse stream', fp, e)
# #         traceback.print_exc()
#         return {}
#     stream_attr = get_stream_attr(stream)
#     if transpose:
#         out_file = Path(str(fp).replace('/midi/midi_sources/', '/midi/transposed/'))
#         transposed_file = transpose_midi2c(fp, stream, out_file=out_file, halfsteps=offset)
#         transposed_stream = music21.converter.parse(fp)
#         t_key = transposed_stream.flat.analyze('key')
#         transposed_attr = {
#             'inferred_keyc': f'{t_key.tonic.name} {t_key.mode}',
#             'midi_keyc': str(transposed_file),
#         }
#         stream_attr = {**stream_attr, **transposed_attr}
#     return stream_attr

def transpose_midi_file(mf, offset):
    for t in mf.tracks:
        if 10 in t.getChannels(): continue # skip percussion
        for e in t.events:
            if e.pitch is None: continue
            e.pitch += offset
    return mf

In [171]:
def compress_midi_file(fp, cutoff=6, unsup_types=set([Track.UNDEF, Track.PERC])):
    music_file = file2mf(fp)
    supported_tracks = []
    for idx,t in enumerate(music_file.tracks):
        track_type = get_track_type(t,idx,fp)
        if len(supported_tracks) >= cutoff: continue
        if track_type in unsup_types: continue
        if track_type in type2inst:
            change_track_instrument(t, type2inst[track_type])
        supported_tracks.append(t)
    music_file.tracks = supported_tracks
    return music_file

In [177]:
music_file = format_midi_file(df.midi_keyc.iloc[10]) # 10 # 7

<MidiEvent PROGRAM_CHANGE, t=None, track=1, channel=1, data=79>
<MidiEvent PROGRAM_CHANGE, t=None, track=2, channel=2, data=0>


In [182]:
def standardize_midi(midi_file, out_file, cutoff=6, offset=None):
    music_file = compress_midi_file(midi_file, cutoff=cutoff) # remove non note tracks and standardize instruments
    music_file = transpose_midi_file(music_file, offset) # transpose to keyc
    s_out = music21.midi.translate.midiFileToStream(music_file) # create music21 stream
    s_comb = music21.instrument.partitionByInstrument(s_out) # combine same track instruments to single part
    return s_comb.write('midi', fp=out_file)

In [183]:
fp = combine_instrument_parts(df.midi_keyc.iloc[10], '/tmp/test10.mid')

<MidiEvent PROGRAM_CHANGE, t=None, track=1, channel=1, data=79>
<MidiEvent PROGRAM_CHANGE, t=None, track=2, channel=2, data=0>


In [184]:
play_midi(fp)

In [51]:
tdf = pd.DataFrame(data={'transcribed_keyc_v1': list(transcribed_files.values())}, index=transcribed_files.keys())

In [52]:
merged_df = df.join(tdf, how='outer')

In [53]:
tdf.shape, df.shape, merged_df.shape

((11531, 1), (12703, 18), (12703, 19))

In [56]:
merged_df.to_csv('hooktheory_metadata_transcribed.csv'); merged_df.head()

,parts,ht_mode,ht_key,original_path,genres,seconds,section,midi_keyc,time_signature,ht_time_signature,ht_bpm,instruments,bpm,midi_title,song_url,artist,inferred_key,inferred_keyc,transcribed_keyc_v1
0,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],15.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,128,"['Piano', 'Piano']",128.0,yu-gi-oh3,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,../data/midi/transcribedc_v1/hooktheory/pianor...
1,"['intro', 'chorus']",1.0,C,../data/midi/hooktheory/pianoroll/w/wayne-shar...,[],25.411765,intro,../data/midi/transposed/hooktheory/pianoroll/w...,3/4,3,85,['Piano'],85.0,yu-gi-oh,https://www.hooktheory.com/theorytab/view/wayn...,wayne-sharpe,C major,C major,NaN
2,['chorus'],6.0,D,../data/midi/hooktheory/pianoroll/w/what-a-day...,['Jazz'],10.000000,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,96,"['Piano', 'Piano']",96.0,kiefer,https://www.hooktheory.com/theorytab/view/what...,what-a-day,A minor,A minor,../data/midi/transcribedc_v1/hooktheory/pianor...
3,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",25.263158,chorus,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...
4,"['verse', 'pre-chorus', 'chorus']",6.0,D,../data/midi/hooktheory/pianoroll/w/whiteflame...,"['J-Pop', 'Pop']",12.631579,verse,../data/midi/transposed/hooktheory/pianoroll/w...,4/4,4,152,"['Piano', 'Piano']",152.0,Senbonzakura,https://www.hooktheory.com/theorytab/view/whit...,whiteflame,D minor,D minor,../data/midi/transcribedc_v1/hooktheory/pianor...


In [26]:
for r in df_filtered.iterrows():
    transcribe_file(r)

KeyError: 139619093464456

In [9]:
def transcribe_file(idxrow):
    idx,row = idxrow
    midi_file = row.midi_keyc
    if not midi_file: return idx,None
    out_file = Path(midi_file.replace('/transposed/', '/transcribed_allcomp/')).with_suffix('.txt')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return idx,str(out_file)
    seq = midi2seq(midi_file)
    string_repr = seq2str(seq)
    with open(out_file, 'w') as tf:
        tf.write(string_repr)
    return idx,str(out_file)